In [1]:
from transitions import Machine

from transitions.extensions.asyncio import AsyncTimeout, AsyncMachine

import enum 
import logging
import asyncio

from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
from pprint import pprint

from datetime import datetime
from math import floor, ceil
import time
from time import sleep

import sys
import os

import zmq
import zmq.asyncio
# from s import Context
import json

logging.basicConfig(level=logging.INFO)

import ccxt
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
client = MongoClient()
db=client.fsmbot
trades = {}

zContext = zmq.asyncio.Context()


In [3]:
exchange = getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})

In [4]:
# t = db.trades.find_one({'_id': ObjectId("615cc81fa0d373f6b44b88b0")})
# t['curStopResp']

In [5]:
# exchange.create_order('ALGO/USDT', 'market', 'buy', 15)
# os.getenv('TEST')

In [6]:
zContext = zmq.asyncio.Context()


states = [
    'ERROR',
    'DISABLED',
    'LIVE',
    'LONG',
    'STOPPED',
    'SOLD'
]
transitions = [
    { 'trigger': 'start', 'source': 'DISABLED', 'dest': 'LIVE'},
    { 'trigger': 'disable', 'source': 'LIVE', 'dest': 'DISABLED'},
    { 'trigger': 'tick', 'source': 'LIVE', 'dest': 'LONG', 'conditions': 'is_buy_signal', 'before' : ['buy_on_exchange', 'load_from_db', 'set_stop_on_exchange']},
    { 'trigger': 'movestop', 'source': 'LONG', 'dest': 'LONG' ,'before': ['save_to_db', 'load_from_db', 'remove_stop_on_exchange', 'set_stop_on_exchange']},
    { 'trigger': 'tick', 'source': 'LONG', 'dest': 'STOPPED', 'conditions': 'is_stopped'},
    { 'trigger': 'tick', 'source': 'LONG', 'dest': 'SOLD' ,'conditions': 'is_sell_signal', 'before': ['remove_stop_on_exchange', 'sell_on_exchange']},
    { 'trigger': 'update', 'source': '*', 'dest': None, 'before': 'save_to_db', 'after': 'load_from_db'},
    { 'trigger': 'load', 'source': '*', 'dest': None, 'before': 'load_from_db'},
    { 'trigger': 'save', 'source': '*', 'dest': None, 'before': 'save_to_db'}

]


class TradeModel(object):
    def __init__(self, data):         
        self.data = data
        self._id = data['_id']
        self.symbol = data['symbol']
        self.machine = Machine(self, states=states, transitions=transitions, send_event=True, initial='DISABLED', 
            after_state_change=['machine_state_changed', 'load_from_db'], on_exception='handle_error')
        self.machine.set_state(data['state'])

        
    def load_from_db(self, event):
        logging.info(event.kwargs)
        # print(f"reloading {self._id}")
        self.data = db.trades.find_one({'_id': ObjectId(self._id)})
        # print(f"reloaded {self.data['_id']} {self.data['symbol']} {self.data['state']}")
        self.symbol = self.data['symbol']
        self.state = self.data['state']
        # print(f"reloaded machine {self.symbol} {self.state}")

    def is_buy_signal(self, event):
        logging.info(event.kwargs)
        ticker = event.kwargs['ticker']
        if float(ticker['curDayClose']) <= float(self.data['trigger']):
            # logging.info(f'+BUY triggered for {self.symbol} / {self.data["_id"]}')
            return True
        # logging.info(f'-BUY triggered for {self.symbol} / {self.data["_id"]}')
        return False

    def is_sell_signal(self, event):
        logging.info(event.kwargs)
        ticker = event.kwargs['ticker']
        if float(ticker['curDayClose']) >= float(self.data['target']):
            # logging.info(f'+SELL triggered for {self.symbol} / {self.data["_id"]}')
            return True
        # logging.info(f'-SELL triggered for {self.symbol} / {self.data["_id"]}')
        return False

    def is_stopped(self, event):
        logging.info(event.kwargs)
        ticker = event.kwargs['ticker']
        if float(ticker['curDayClose']) <= float(self.data['stop']):
            # logging.info(f'+BUY triggered for {self.symbol} / {self.data["_id"]}')
            return True
        # logging.info(f'-BUY triggered for {self.symbol} / {self.data["_id"]}')
        return False


    def buy_on_exchange(self, event):
        logging.info(event.kwargs)
        try:
            resp = exchange.create_order(self.symbol, 'market', 'buy', float(self.data["qty"]));
            db.trades.update_one({'_id' : self.data['_id'] }, {
            '$set': {
                'buyResp': resp
            }})
        except BaseException as err:
            logging.error(err)
            db.trades.update_one({'_id' : self.data['_id'] }, {
            '$set': {
                'state': 'ERROR',
                'err': str(err)
            }})

    def sell_on_exchange(self, event):
        logging.info(event.kwargs)
        try:
            available_qty = self.data["buyResp"]['filled'] - self.data["buyResp"]['fee']['cost']
            resp = exchange.create_order(self.symbol, 'market', 'sell', available_qty)
            db.trades.update_one({'_id' : self.data['_id'] }, {
            '$set': {
                'sellResp': resp
            }})
        except BaseException as err:
            logging.error(err)
            db.trades.update_one({'_id' : self.data['_id'] }, {
            '$set': {
                'state': 'ERROR',
                'err': str(err)
            }})

    # def update_stop(self, event):
    #     logging.info(event.kwargs)
    #     try:
    #         db.trades.update_one({'_id' : self.data['_id'] }, {
    #             '$set': {
    #                 'stop': event.kwargs['stop']
    #             }})
    #     except BaseException as err:
    #         logging.error(err)
    #         db.trades.update_one({'_id' : self.data['_id'] }, {
    #         '$set': {
    #             'state': 'ERROR',
    #             'err': err
    #         }})

    def save_to_db(self, event):
        logging.info(event.kwargs)
        try:
            db.trades.update_one({'_id' : self.data['_id'] }, event.kwargs['data'] )
        except BaseException as err:
            logging.error(err)
            db.trades.update_one({'_id' : self.data['_id'] }, {
            '$set': {
                'state': 'ERROR',
                'err': str(err)
            }})

    def set_stop_on_exchange(self, event):
        logging.info(event.kwargs)
        try:
            available_qty = self.data["buyResp"]['filled'] - self.data["buyResp"]['fee']['cost']
            resp = exchange.create_order(self.symbol, 'STOP_LOSS_LIMIT', 'sell', available_qty * 0.99, 
                0.90 * float(self.data["stop"]), {'stopPrice': float(self.data["stop"]),'type': 'stopLimit'})
            db.trades.update_one({'_id' : self.data['_id'] }, {
            '$set': {
                'curStopResp': resp
            }})
        except BaseException as err:
            logging.error(err)
            db.trades.update_one({'_id' : self.data['_id'] }, {
            '$set': {
                'state': 'ERROR',
                'err': str(err)
            }})
        # self.data['curStopResp'] = resp

    def remove_stop_on_exchange(self, event):
        logging.info(event.kwargs)
        try:
            if self.data['curStopResp']:
                resp = exchange.cancel_order(self.data['curStopResp']['id'], symbol=self.symbol)
                db.trades.update_one({'_id' : self.data['_id'] }, {
                    '$set': {                        
                        'cancelStopResp': resp
                    }})
        
        except BaseException as err:
            logging.error(err)
            db.trades.update_one({'_id' : self.data['_id'] }, {
            '$set': {
                'state': 'ERROR',
                'err': str(err)
            }})
      

    def machine_state_changed(self, event):
        logging.info(event.kwargs)
        db.trades.update_one({'_id' : self.data['_id'] }, {
        '$set': {
            'state': 'ERROR',
            'state': self.state
          }})

    def handle_error(self, event):
        logging.info(event.kwargs)
        db.trades.update_one({'_id' : self.data['_id'] }, {
        '$set': {
            'state': 'ERROR',
            'error': { 
                'error': event.error,
                'kwargs': event.kwargs 
                }
          }})


In [7]:
def add_trade_machine(trade):
    if not trade['symbol'] in trades: trades[trade['symbol']] = {}
    model = TradeModel(trade)
    trades[trade['symbol']][str(trade['_id'])] = model
    return model

def load_trades_from_db():
    for trade in db.trades.find():
        try:
            add_trade_machine(trade)
        except BaseException as err:
            logging.error(err)
            


In [8]:
load_trades_from_db()

In [23]:
# trades
model = [*trades['ALGOUSDT'].values()][0]
# model.update(data={'$set': {'curStopResp': None}})
model.movestop(data={'$set': {'stop': 1.78}})
# model.machine
# model.movestop(stop= 1.6)
# tradeModel.movestop(stop=1.7930)
# [*trades['ALGOUSDT'].values()]
# str(ObjectId('615e6a55022458a789d8cb84'))

INFO:root:{'data': {'$set': {'curStopResp': None}}}
INFO:transitions.core:Executed callback 'save_to_db'
INFO:root:{'data': {'$set': {'curStopResp': None}}}
INFO:transitions.core:Executed callback 'load_from_db'
INFO:root:{'data': {'$set': {'curStopResp': None}}}
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:root:{'data': {'$set': {'curStopResp': None}}}
INFO:transitions.core:Executed callback 'load_from_db'


True

load_trades_from_tb()

In [10]:
async def run_ticker():
    socket = zContext.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

    while True:
        tickerString = await socket.recv_string()
        _, symbol, ticker = tickerString.split()
        ticker = json.loads(ticker)
        if symbol in trades:        
            for model in trades[symbol].values():
                if model.state in ['LIVE', 'LONG']:
                    model.tick(ticker=ticker)
                    

In [11]:
# async def another_thread_coro():
#     while True:
#         print('another_thread_coro', datetime.now())
#         await asyncio.sleep(1);

# await another_thread_coro()

In [12]:
from threading import Thread

ticker_loop = asyncio.new_event_loop()

def setup_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

ticker_thread = Thread(target=setup_loop, args=(ticker_loop,))
ticker_thread.start()
ticker_future = asyncio.run_coroutine_threadsafe(run_ticker(), ticker_loop)

In [13]:
def createTrade(symbol, state, qty, trigger, stop, target):
    datecreated = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    tradeRes = db.trades.insert_one({
            "id": f"{symbol}_{datecreated}",
            "datecreated" : datecreated,
            "state": state,
            "symbol" : symbol,
            "trigger": trigger,
            "stop": stop,
            "target": target,
            "qty": qty
        })
    trade = db.trades.find_one({'_id': tradeRes.inserted_id})
    # add_trade_machine(trade)
    return trade

In [14]:
# (capital, risk) = 50000, 0.01
# symbol, state = 'ALGOUSDT', 'DISABLED'
# (trigger, stop, target) = 1.764, 1.706, 1.923
# qty = (capital * risk) / (trigger - stop) 
# size = qty * trigger

# # trade = createTrade(symbol, state, qty, trigger, stop, target)
# # (tradeModel, tradeMachine) = add_trade_machine(trade)

# # tradeModel.start()
# # tradeModel
# qty,size

In [15]:
# (capital, risk) = 50000, 0.01
# symbol, state = 'ALGOUSDT', 'DISABLED'
# (trigger, stop, target) = 1.7531, 1.7715, 1.9914
# qty = (capital * risk) / (trigger - stop) 
# size = qty * trigger

# trade = createTrade(symbol, state, qty, trigger, stop, target)
# (tradeModel, tradeMachine) = add_trade_machine(trade)

# tradeModel.start()
# tradeModel

In [16]:

# symbol, state = 'ALGOUSDT', 'DISABLED'
# (trigger, stop, target) = 2, 1.7735, 2.20
# trade = createTrade(symbol, state, 15, trigger, stop, target)
# (tradeModel, tradeMachine) = add_trade_machine(trade)

# tradeModel.start()
# tradeModel

In [17]:
# trades = {} 
# load_trades_from_db()
# trades

In [18]:
# # trades
# for symbol, trade_chunk_arr in trades.items():
#     for chunk in trade_chunk_arr:
#         model = chunk['model'] 
#         machine = chunk['machine']
#         if model.state != 'LONG':
#             continue
#         trade = model.trade
#         stop_order = exchange.fetchOrder(trade['curStopResp']['id'], symbol=symbol)
#         if exchange.fetchOrderStatus(trade['curStopResp']['id'], symbol='ALGOUSDT') == 'closed':
#             print(trade['id'], 'closed')
#         # print(chunk)
#     # print(tradeChunk)
# # tradeModel.trade